In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cats-and-dogs/val.csv
/kaggle/input/cats-and-dogs/train.csv
/kaggle/input/cats-and-dogs/val/classname.txt
/kaggle/input/cats-and-dogs/val/dog/havanese_144_jpg.rf.d3e2e8cb9a7801e61894e372c4f1e1bb.jpg
/kaggle/input/cats-and-dogs/val/dog/american_pit_bull_terrier_135_jpg.rf.f23ba7a8a1017fc71e62dcdeb3af9fa9.jpg
/kaggle/input/cats-and-dogs/val/dog/great_pyrenees_190_jpg.rf.ebd07e7aae6a416425fa1f3c89872b61.jpg
/kaggle/input/cats-and-dogs/val/dog/great_pyrenees_144_jpg.rf.d416d150a51cd8c6ae1489957c49e01b.jpg
/kaggle/input/cats-and-dogs/val/dog/chihuahua_17_jpg.rf.d97aa2c7e61de5128686d82b87c95215.jpg
/kaggle/input/cats-and-dogs/val/dog/english_cocker_spaniel_17_jpg.rf.f22312874dde3803847df16bcfe12f83.jpg
/kaggle/input/cats-and-dogs/val/dog/american_bulldog_148_jpg.rf.f0a7b86489589aa05534a6f8cb0fe70a.jpg
/kaggle/input/cats-and-dogs/val/dog/english_cocker_spaniel_121_jpg.rf.db065f7b515f83644b2fcb7525073d0b.jpg
/kaggle/input/cats-and-dogs/val/dog/pomeranian_134_jpg.rf.fb1b528eaddde0

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,BatchNormalization ,Flatten , MaxPooling2D,Conv2D ,Dropout
from keras.models import Sequential 
from keras.preprocessing import image

from tensorflow.keras.preprocessing.image import ImageDataGenerator

2026-01-20 13:50:23.571734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768917023.901904      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768917023.991667      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768917024.736656      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768917024.736716      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768917024.736719      17 computation_placer.cc:177] computation placer alr

In [3]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/kaggle/input/cats-and-dogs/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256),
    shuffle = True 
)

val_ds = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/cats-and-dogs/val',
    labels='inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256),
    shuffle = False
)

Found 275 files belonging to 2 classes.
Found 70 files belonging to 2 classes.


2026-01-20 13:50:45.606178: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
#Normalise 
def process(image,label):
    image = tf.cast(image/255,tf.float32)
    return image,label

train_ds = train_ds.map(process)
val_ds = val_ds.map(process)

In [5]:
import os 
import cv2
import random

In [6]:
save_dir = '/kaggle/working/augumentation'
os.makedirs(save_dir,exist_ok=True)

In [7]:
base_dir = "/kaggle/input/cats-and-dogs/train"
categories = ["cat", "dog"]

data = []

for i in categories:
    folder_path = os.path.join(base_dir, i)
    label = 0 if i == "cat" else 1

    for j in os.listdir(folder_path):
        img_path = os.path.join(folder_path, j)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img = cv2.resize(img, (150, 150))
        data.append([img, label])


In [8]:
random.shuffle(data)

In [9]:
x=[]
y=[]
for i in data:
    x.append(i[0])
    y.append(i[1])

In [10]:
y = np.array(y)
X = np.array(x)

In [11]:
X.shape

(275, 150, 150, 3)

In [12]:
X=X/255

In [13]:
from keras  import Sequential 
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Activation,Dropout

In [14]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64,activation='relu')) #if no activation-> just linear
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
          

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
X.shape , y.shape

((275, 150, 150, 3), (275,))

In [16]:
model.fit(X ,y ,epochs=5,validation_split=0.1)

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 550ms/step - accuracy: 0.6022 - loss: 0.6763 - val_accuracy: 0.6786 - val_loss: 0.6301
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 519ms/step - accuracy: 0.6686 - loss: 0.6404 - val_accuracy: 0.6786 - val_loss: 0.6153
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 517ms/step - accuracy: 0.6588 - loss: 0.6162 - val_accuracy: 0.6786 - val_loss: 0.6235
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 503ms/step - accuracy: 0.6058 - loss: 0.6388 - val_accuracy: 0.6786 - val_loss: 0.6024
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 487ms/step - accuracy: 0.6463 - loss: 0.6313 - val_accuracy: 0.6786 - val_loss: 0.6110


# highest validation accuracy achieved = 0.5714

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
batch_size=16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range = 0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/cats-and-dogs/train',
    target_size = (150,150),
    batch_size=batch_size,
    class_mode = 'binary'
)

validation_generator=test_datagen.flow_from_directory(
    '/kaggle/input/cats-and-dogs/val',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 275 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


In [19]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64,activation='relu')) #if no activation-> just linear
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
          

In [20]:
model.fit(
    train_generator,
    steps_per_epoch=275//batch_size,
    epochs = 25,
    validation_data = validation_generator,
    validation_steps = 70//batch_size
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 330ms/step - accuracy: 0.5584 - loss: 0.6696 - val_accuracy: 0.6406 - val_loss: 0.6690
Epoch 2/25
 1/17 ━━━━━━━━━━━━━━━━━━━━ 4s 255ms/step - accuracy: 0.6875 - loss: 0.6295

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6875 - loss: 0.6295 - val_accuracy: 0.6250 - val_loss: 0.6618
Epoch 3/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 300ms/step - accuracy: 0.6325 - loss: 0.6573 - val_accuracy: 0.6406 - val_loss: 0.7131
Epoch 4/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5625 - loss: 0.6595 - val_accuracy: 0.6562 - val_loss: 0.6595
Epoch 5/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 303ms/step - accuracy: 0.6530 - loss: 0.6658 - val_accuracy: 0.6562 - val_loss: 0.6166
Epoch 6/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5625 - loss: 0.6495 - val_accuracy: 0.6875 - val_loss: 0.6079
Epoch 7/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 337ms/step - accuracy: 0.6203 - loss: 0.6235 - val_accuracy: 0.6562 - val_loss: 0.6332
Epoch 8/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6875 - loss: 0.6366 - val_accuracy: 0.6719 - val_loss: 0.6291
Epoch 9/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 304ms/step - accuracy: 0.6641 - loss: 0.5874 - val_accuracy: 0.6562 - val_los

# highest validation accuracy achieved = 0.7031